In [15]:
import nibabel as nib
import numpy as np
import glob
from skimage.filters import threshold_otsu
from cupyx.scipy import ndimage
import cupy as cp
from skimage.morphology import ball



path_MRI = '/mnt/data/psteinmetz/neotex/to_process/Ozgun/*/RawVolume/*.nii.gz'
path_tumors = '/mnt/data/psteinmetz/neotex/to_process/Ozgun/*/RoiVolume/VOI.nii.gz'
tumor_mask_files = glob.glob(path_tumors)
tumor_files = glob.glob(path_MRI)
with cp.cuda.Device(2):
    for tumor_file, tumor_mask_file in zip(tumor_files, tumor_mask_files):
        print(tumor_file)
        # Load the tumor mask nifti file
        mri_nii = nib.load(tumor_file)
        tumor_mask_nii = nib.load(tumor_mask_file)
        mri_data = mri_nii.get_fdata()
        tumor_mask_data = tumor_mask_nii.get_fdata()
        
        # Define a structuring element (disk size can be adjusted based on the gaps)
        structuring_element = cp.array(ball(4))


        
        # If the tumor mask is larger, crop it to match the MRI shape
        if tumor_mask_data.shape != mri_data.shape:
            print('mismatch files')
            tumor_mask_data = tumor_mask_data[..., :mri_data.shape[2]]
        
        # Extract the ROI from the MRI using the tumor mask
        mri_roi = mri_data[tumor_mask_data > 0]

        # Calculate Otsu's threshold based on the MRI values within the tumor mask
        otsu_threshold = threshold_otsu(mri_roi)

        # Apply the threshold to the MRI within the tumor mask
        refined_mask = cp.array(np.zeros_like(mri_data))
        refined_mask[(mri_data >= otsu_threshold) & (tumor_mask_data > 0)] = 1
        
        # Apply morphological closing to close gaps in the mask
        closed_mask = cp.asnumpy(ndimage.binary_closing(refined_mask, structure=structuring_element))
        # Assuming 'closed_mask' is your resulting closed tumor mask (after morphological closing)
        closed_mask_nii = nib.Nifti1Image(closed_mask.astype(np.uint8), affine=tumor_mask_nii.affine)  # Use an appropriate affine if needed

        # Save the NIfTI file
        nib.save(closed_mask_nii, tumor_mask_file[:-7] + '_morphoclosing.nii.gz')

        # Calculate the Otsu threshold as a percentage of the maximum intensity in the ROI
        max_intensity = np.max(mri_roi)
        otsu_threshold_percentage = (otsu_threshold / max_intensity) * 100

        print(f'Otsu threshold: {otsu_threshold} (which is {otsu_threshold_percentage:.2f}% of the max intensity {max_intensity})')
        # Find the bounding box around the tumor
        # non_zero_coords = np.argwhere(tumor_mask_data > 0) # Find all non-zero points in the mask
        # min_coords = non_zero_coords.min(axis=0)  # Minimum corner of the bounding box
        # max_coords = non_zero_coords.max(axis=0)  # Maximum corner of the bounding box
        
        # # Debugging: Print the bounding box coordinates and tumor mask shape
        # print(f"Tumor mask shape: {tumor_mask_data.shape}")
        # print(f"Bounding box min coords: {min_coords}")
        # print(f"Bounding box max coords: {max_coords}")

        # # Create a new empty mask of the same shape as the original
        # bounding_box_mask = np.zeros_like(tumor_mask_data)

        # # Set the bounding box region to 1 (or another value if needed)
        # bounding_box_mask[min_coords[0]:max_coords[0]+1,
        #                 min_coords[1]:max_coords[1]+1,
        #                 min_coords[2]:max_coords[2]+1] = 1
        
        # # Debugging: Check if the bounding box was applied correctly
        # bounding_box_voxels = np.argwhere(bounding_box_mask == 1)
        # print(f"Number of bounding box voxels: {bounding_box_voxels.shape[0]}")
        # print(f"Unique values in bounding box mask: {np.unique(bounding_box_mask)}")

        # # Create a new NIfTI image from the bounding box mask
        # bounding_box_nii = nib.Nifti1Image(bounding_box_mask, tumor_mask_nii.affine, tumor_mask_nii.header)

        # # Save the new bounding box mask
        # bounding_box_file = tumor_mask_file[:-7] + '_bb.nii.gz'
        # nib.save(bounding_box_nii, bounding_box_file)

        # print(f"Bounding box mask saved to: {bounding_box_file}")



/mnt/data/psteinmetz/neotex/to_process/Ozgun/221/RawVolume/221_subtracted_resampled_Bspline_zscore.nii.gz
Otsu threshold: 6.3167035835795105 (which is 22.99% of the max intensity 27.47316551208496)
/mnt/data/psteinmetz/neotex/to_process/Ozgun/197/RawVolume/197_subtracted_resampled_Bspline_zscore.nii.gz
Otsu threshold: 1.8954176669940352 (which is 24.49% of the max intensity 7.738675594329834)
/mnt/data/psteinmetz/neotex/to_process/Ozgun/220/RawVolume/220_subtracted_resampled_Bspline_zscore.nii.gz
Otsu threshold: 1.4890067484229803 (which is 21.58% of the max intensity 6.900973320007324)
/mnt/data/psteinmetz/neotex/to_process/Ozgun/188/RawVolume/188_subtracted_resampled_Bspline_zscore.nii.gz
Otsu threshold: 5.5239850981161 (which is 27.21% of the max intensity 20.30360221862793)
/mnt/data/psteinmetz/neotex/to_process/Ozgun/213/RawVolume/213_subtracted_resampled_Bspline_zscore.nii.gz
Otsu threshold: 5.014182538725436 (which is 23.66% of the max intensity 21.19698143005371)
/mnt/data/pste

In [2]:
tumor_files

['/mnt/data/psteinmetz/neotex/to_process/Ozgun/221/RawVolume/221_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/197/RawVolume/197_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/220/RawVolume/220_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/188/RawVolume/188_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/213/RawVolume/213_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/190/RawVolume/190_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/181/RawVolume/181_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/219/RawVolume/219_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/data/psteinmetz/neotex/to_process/Ozgun/167/RawVolume/167_subtracted_resampled_Bspline_zscore.nii.gz',
 '/mnt/dat